**Keras Tuner uses three hyperparameter tuning techniques:**
- Randomsearch
- Hyperband
- BayesianOptimization
- Sklearn for sckit-learn models

In [1]:
# globally useful imports of standard libraries needed in this notebook
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os.path
from datetime import datetime

# specific libraries or classes needed for the work in this notebook
#from sklearn.linear_model import LogisticRegression
#from sklearn.linear_model import LogisticRegressionCV
#from sklearn.svm import SVC
#from sklearn.naive_bayes import GaussianNB
#from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, f_classif, chi2, SelectFpr, f_regression, mutual_info_classif
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFE
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import ExtraTreesClassifier
#from sklearn.pipeline import Pipeline
# need to use Pipeline from imblearn to add in a downsample or upsample
# to cross validation training
from imblearn.pipeline import Pipeline

# import project specific modules used in this notebook
import sys
sys.path.append('../src')
import mindwandering.data
import mindwandering.features
import mindwandering.evaluate
from mindwandering.data import FeatureScalerTransformer
from mindwandering.data import WinsorizationOutlierTransformer
from mindwandering.data import VIFThresholdTransformer
from mindwandering.data import FeatureSelectionTransformer
from mindwandering.data import ClassImbalanceTransformer
from mindwandering.data import GridSearchProgressHack

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.layers import Input, Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import SGD,Adam,RMSprop
from tensorflow.keras.losses import binary_crossentropy

from kerastuner.tuners import Hyperband,BayesianOptimization
import kerastuner
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

In [3]:
# Load Data and Features
df_features = mindwandering.data.get_df_features()
mind_wandered_label = mindwandering.data.get_mind_wandered_label()
participant_ids = mindwandering.data.get_participant_ids()

In [4]:
inputs_shape=df_features.shape[1]

In [5]:
import tensorflow
auc_roc = tensorflow.keras.metrics.AUC(name='auc_roc',num_thresholds=20000,curve='ROC')

In [6]:
from sklearn.model_selection import train_test_split
# split into train test sets
X_train, X_test, y_train, y_test = train_test_split(df_features, mind_wandered_label, test_size=0.20,random_state=123,stratify=mind_wandered_label)

In [7]:
import kerastuner as kt
import tensorflow as tf

def build_model(hp):
    '''This function will build a model in order to search for three Hyperparameters 
    number of layers between 1 to 20;
    four activation functions namely relu,selu,tanh,elu;
    Adam optimizers learning rate: 1e-2,1e-3'''
    model = tf.keras.Sequential()
    for i in range(hp.Int('layers', 1, 5)):
        model.add(tf.keras.layers.Dense(
            units=hp.Int('units_' + str(i), 32, 512, step=32),
            activation=hp.Choice('act_' + str(i), ['relu', 'tanh','selu','elu','swish'])))
        drop_rate = hp.Choice('drop_rate',[0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])
        model.add(keras.layers.Dropout(rate=drop_rate))
        #model.add(tf.keras.layers.Dropout(dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.5)))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),loss='binary_crossentropy', metrics=['accuracy',auc_roc])
    return model

# RandomSearch

In [8]:
tuner = RandomSearch(
    build_model,
    objective=kerastuner.Objective('auc_roc', direction='max'),
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir13',
    project_name='Keras-tuner-May-5-2021')

In [9]:
tuner.search_space_summary()

Search space summary
Default search space size: 5
layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 5, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
act_0 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'selu', 'elu', 'swish'], 'ordered': False}
drop_rate (Choice)
{'default': 0.0, 'conditions': [], 'values': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], 'ordered': True}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001], 'ordered': True}


In [10]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 13s]
auc_roc: 0.5117804010709127

Best auc_roc So Far: 0.549515942732493
Total elapsed time: 00h 00m 57s
INFO:tensorflow:Oracle triggered exit


In [11]:
tuner.results_summary()

Results summary
Results in my_dir13/Keras-tuner-May-5-2021
Showing 10 best trials
Objective(name='auc_roc', direction='max')
Trial summary
Hyperparameters:
layers: 2
units_0: 256
act_0: swish
drop_rate: 0.0
learning_rate: 0.001
units_1: 32
act_1: relu
Score: 0.549515942732493
Trial summary
Hyperparameters:
layers: 1
units_0: 224
act_0: tanh
drop_rate: 0.8
learning_rate: 0.001
units_1: 288
act_1: tanh
units_2: 96
act_2: selu
units_3: 384
act_3: elu
units_4: 384
act_4: selu
Score: 0.5253549218177795
Trial summary
Hyperparameters:
layers: 4
units_0: 192
act_0: selu
drop_rate: 0.4
learning_rate: 0.01
units_1: 224
act_1: swish
units_2: 288
act_2: swish
units_3: 384
act_3: elu
units_4: 416
act_4: relu
Score: 0.5153156518936157
Trial summary
Hyperparameters:
layers: 5
units_0: 96
act_0: selu
drop_rate: 0.4
learning_rate: 0.01
units_1: 416
act_1: tanh
units_2: 32
act_2: relu
units_3: 32
act_3: relu
units_4: 32
act_4: relu
Score: 0.5119073788324991
Trial summary
Hyperparameters:
layers: 3
units

# Hyperband

In [12]:
# Construct the Hyperband tuner using the hypermodel class created
tuner = Hyperband(
    build_model,
    objective=kerastuner.Objective('auc_roc', direction='max'),
    max_epochs=10,
    seed=10,
    directory='my_dir14',
    project_name='Keras-tuner-May-5-2021-2:35am')


# Search for the best parameters of the neural network using the contructed Hypberband tuner
tuner.search(X_train.values, y_train.values.flatten(),
             epochs=10,
             validation_data=(X_test.values, y_test.values.flatten()))

# Get the best hyperparameters from the search
params = tuner.get_best_hyperparameters()[0]

Trial 30 Complete [00h 00m 06s]
auc_roc: 0.511381983757019

Best auc_roc So Far: 0.5470007061958313
Total elapsed time: 00h 01m 33s
INFO:tensorflow:Oracle triggered exit


In [13]:
tuner.results_summary()

Results summary
Results in my_dir14/Keras-tuner-May-5-2021-2:35am
Showing 10 best trials
Objective(name='auc_roc', direction='max')
Trial summary
Hyperparameters:
layers: 4
units_0: 384
act_0: tanh
drop_rate: 0.1
learning_rate: 0.001
units_1: 64
act_1: tanh
units_2: 256
act_2: elu
units_3: 480
act_3: swish
units_4: 384
act_4: relu
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 8509da899df01310d55e9846b246f490
Score: 0.5470007061958313
Trial summary
Hyperparameters:
layers: 4
units_0: 384
act_0: tanh
drop_rate: 0.1
learning_rate: 0.001
units_1: 64
act_1: tanh
units_2: 256
act_2: elu
units_3: 480
act_3: swish
units_4: 384
act_4: relu
tuner/epochs: 4
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.5395991206169128
Trial summary
Hyperparameters:
layers: 2
units_0: 416
act_0: tanh
drop_rate: 0.2
learning_rate: 0.001
units_1: 288
act_1: tanh
units_2: 256
act_2: selu
units_3: 128
act_3: relu
units_4: 96
act_4: elu
tuner/epochs: 10
tuner

# BayesianOptimization

In [14]:
tuner=BayesianOptimization(
    build_model,
    objective=kerastuner.Objective('auc_roc', direction='max'),
    max_trials=4,
    directory='my_dir15',
    project_name='Keras-tuner-May-5-2021-2:36am')

# Search for the best parameters of the neural network using the contructed Hypberband tuner
tuner.search(X_train.values, y_train.values.flatten(),
             epochs=10,
             validation_data=(X_test.values, y_test.values.flatten()))

# Get the best hyperparameters from the search
params = tuner.get_best_hyperparameters()[0]

Trial 4 Complete [00h 00m 04s]
auc_roc: 0.5543853640556335

Best auc_roc So Far: 0.5543853640556335
Total elapsed time: 00h 00m 20s
INFO:tensorflow:Oracle triggered exit


In [15]:
tuner.results_summary()

Results summary
Results in my_dir15/Keras-tuner-May-5-2021-2:36am
Showing 10 best trials
Objective(name='auc_roc', direction='max')
Trial summary
Hyperparameters:
layers: 1
units_0: 192
act_0: tanh
drop_rate: 0.1
learning_rate: 0.001
units_1: 96
act_1: elu
units_2: 480
act_2: swish
Score: 0.5543853640556335
Trial summary
Hyperparameters:
layers: 3
units_0: 480
act_0: tanh
drop_rate: 0.8
learning_rate: 0.001
units_1: 32
act_1: relu
units_2: 32
act_2: relu
Score: 0.5213000774383545
Trial summary
Hyperparameters:
layers: 2
units_0: 256
act_0: relu
drop_rate: 0.4
learning_rate: 0.001
units_1: 64
act_1: tanh
units_2: 224
act_2: tanh
Score: 0.52048659324646
Trial summary
Hyperparameters:
layers: 3
units_0: 224
act_0: tanh
drop_rate: 0.9
learning_rate: 0.001
units_1: 224
act_1: tanh
units_2: 288
act_2: elu
Score: 0.5163724422454834
